# Collecting Top-100 Songs

They must be at the top for a reason... So in order to test that theory we need to download them.

<hr>

First we are going to pull the year end top 100 from 2006-2020 from billboard

For some reason, searching between 1991 to 2005 only pulls results from 2006, so we'll have to search elsewhere for 1990-2005.

### !!! Update !!! 

I wrote a good loop to collect songs in a list, but it trips up because 2011 only has 99 in its top-100. Apparently that is due to Ceelo Green's F*** You not being displayed at number 7, 2011 will have to be updated accordingly

### !!! Update 2 !!! 

Apparently another song has been removed from the list. So now we'll have to find what year that song was in.

In [83]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

top100 = []
year = 2006
for i in range(15):
    url = 'https://www.billboard.com/charts/year-end/' + str(year) +'/hot-100-songs'
    response = requests.get(url)
    
    soup = BeautifulSoup(response.text)
    
    artistLists = soup.find_all("div", {"class": "ye-chart-item__artist"})
    titleLists = soup.find_all("div", {"class": "ye-chart-item__title"})
    
    a = 0
    for i in range(len(artistLists)):
        if year == 2011 and i == 6:
            top100.append([2011,i + 1,"CeeLo Green","Forget You"])
            a = a+1
        if year == 2016 and i == 86:
            top100.append([2016,i + 1,"Fat Joe, Remy Ma and Jay-Z featuring French Montana and Infared","All the Way Up"])
            a = a+1
        
        a = a + 1 #rank
        b = year #year
        c =  artistLists[i].text.replace("\n", "") #artist
        d =  titleLists[i].text.replace("\n", "") #song name
        top100.append([b,a,c,d])
    
    year = year+1 


In [87]:
# this block was used for error testing
# fixed by checking song position for two problem songs in list

'''
from collections import Counter

top100[0][0]

yearList = []
for i in range(len(top100)):    
    yearList.append(top100[i][0])
    
c = Counter(yearList)
c

# in 2016, 87th is missing this is "All The Way Up"
'''

'\nfrom collections import Counter\n\ntop100[0][0]\n\nyearList = []\nfor i in range(len(top100)):    \n    yearList.append(top100[i][0])\n    \nc = Counter(yearList)\nc\n\n# in 2016, 87th is missing this is "All The Way Up"\n'

In [86]:
# test the two problem areas (#7 in 2011, #87 in 2016)

print(top100[505:510])
print(top100[1085:1090])

#both look good, so its time to save this work

[[2011, 6, 'Bruno Mars', 'Grenade'], [2011, 7, 'CeeLo Green', 'Forget You'], [2011, 8, 'Nicki Minaj', 'Super Bass'], [2011, 9, 'Maroon 5 Featuring Christina Aguilera', 'Moves Like Jagger'], [2011, 10, 'The Black Eyed Peas', "Just Can't Get Enough"]]
[[2016, 86, 'Shawn Mendes & Camila Cabello', 'I Know What You Did Last Summer'], [2016, 87, 'Fat Joe, Remy Ma and Jay-Z featuring French Montana and Infared', 'All the Way Up'], [2016, 88, 'Silento', 'Watch Me'], [2016, 89, 'Chris Brown', 'Back To Sleep'], [2016, 90, 'Usher Featuring Young Thug', 'No Limit']]


In [91]:
from sqlalchemy import create_engine
from pandas.io import sql
import pandas as pd 

hostname=''
dbname=''
uname=''
pwd=''

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=dbname, user=uname, pw=pwd))

df = pd.DataFrame(top100, columns = ['year', 'rank','artist','title'])

df.to_sql('top100', engine, if_exists='append', index=False)

### Now that we got 2006-2020, we have to grab 1990-2005. Wikipedia seems to have a good list to grab from

In [132]:
# start with an initial 1990, and append 91-05
df = pd.read_html("https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_1990")[0]
df['year'] = 1990

year = 1991
for i in range(14):
    url = "https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_" + str(year)
    df2 = pd.read_html(url)[0]
    df2['year'] = year
    df = df.append(df2)
    
    year = year + 1

In [118]:
# all the songs are in order, so
# to eliminate the two different 
# ranks we can just iterate
# but we have to make sure theres 
# 100 for each year first

year = 1990
for i in range(15):
    print(len(df[df['year'].isin([year])]), year)
    year = year+1

100 1990
100 1991
100 1992
100 1993
100 1994
100 1995
100 1996
100 1997
100 1998
100 1999
100 2000
100 2001
100 2002
100 2003
100 2004


In [133]:
df['rank'] = None
rank = 1
df = df.reset_index()
for i, row in df.iterrows():
    df.at[i,'rank'] = rank
    if rank == 100:
        rank = 1
    else:
        rank = rank + 1

In [134]:
df

,index,No.,Title,Artist(s),year,№,rank
0,0,1.0,"""Hold On""",Wilson Phillips,1990,NaN,1
1,1,2.0,"""It Must Have Been Love""",Roxette,1990,NaN,2
2,2,3.0,"""Nothing Compares 2 U""",Sinéad O'Connor,1990,NaN,3
3,3,4.0,"""Poison""",Bell Biv DeVoe,1990,NaN,4
4,4,5.0,"""Vogue""",Madonna,1990,NaN,5
...,...,...,...,...,...,...,...
1495,95,96.0,"""Over and Over""",Nelly featuring Tim McGraw,2004,NaN,96
1496,96,97.0,"""Redneck Woman""",Gretchen Wilson,2004,NaN,97
1497,97,98.0,"""Just Lose It""",Eminem,2004,NaN,98
1498,98,99.0,"""Letters from Home""",John Michael Montgomery,2004,NaN,99


### Perfect, now we just have to clean up the dataframe, make sure the column names line up, and then its ready to go to the database

In [136]:
# dropping the stuff we dont need

df = df.drop(['index', 'No.','№'], axis=1)

In [137]:
# relabeling columns

df.columns = ['title', 'artist','year','rank']

In [141]:
# removing quotes from the song title

for i, row in df.iterrows():
    titleStrip = df.at[i,'title'][1:-1]
    df.at[i,'title'] = titleStrip

In [144]:
# reorder the dataframe to match the last one just in case

df = df[['year', 'rank','artist','title']]

In [146]:
# now we can append it to the table we just made

df.to_sql('top100', engine, if_exists='append', index=False)

Okay, now the easy part is over. Now we have to collect all the spotify URIs for the songs we just collected.

### !!! Update !!!

it appears that the long artist names when others are featured caused for a lot of songs to be missed (almost half!), so a quick fix appears to be just using the first word out of the artist's name.

### !!! Update 2 !!! 

that actually performed just as poorly, but the results combined leave me at 900/3000 missing, so lets see what else we can do

### !!! Update 3 !!!

I was actually searching for ONLY one parameter instead of artist AND song becausue i used a comma to concat instead of a + which turned the string into a tuple, the next go around should work the best

In [147]:
top100 = pd.read_sql('SELECT * FROM top100', con=engine)

In [171]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials #To access authorised Spotify data

client_id = ''
client_secret = ''
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) #spotify object to access API


In [243]:
uriList = []
for i in top100.iterrows():
    # get name of search query
    artist = i[1]['artist'].split()[0]
    song = i[1]['title']
    name = " artist:" + artist + " track:" + song
    result = sp.search(name,type="track", limit=50)
    
    # reset foundURI flag and search results for song
    foundURI = False
    for i in range(len(result['tracks']['items'])):
        if result['tracks']['items'][i]['name'][:6] == song[:6]:
            uri = result['tracks']['items'][i]['uri']
            foundURI = True
            break
            
    # if found append URI
    if foundURI:
        uriList.append(uri)
    else:
        uriList.append("-1")

In [265]:
uris = pd.DataFrame(uriList,columns=['uri'])
top100['uri'] = uris['uri'].values

In [274]:
leftovers = top100[(top100 == '-1').any(axis=1)]

C:\Users\Ambrose\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


### Masking it out reveals that there are still some stragglers. Tweaking the parameters of the search may yield more results.

In [275]:
leftovers

,year,rank,artist,title,uri
23,2006,24,Lil Jon Featuring E-40 & Sean Paul Of The Youn...,Snap Yo Fingers,-1
31,2006,32,Christina Aguilera,Ain't No Other Man,-1
40,2006,41,T-Pain Featuring Mike Jones,I'm N Luv (Wit A Stripper),-1
42,2006,43,Fort Minor Featuring Holly Brook,Where'd You Go,-1
49,2006,50,Mariah Carey,Don't Forget About Us,-1
...,...,...,...,...,...
2870,2003,71,Frankie J,Don't Wanna Try,-1
2884,2003,85,Young Gunz,"Can't Stop, Won't Stop",-1
2896,2003,97,Norah Jones,Don't Know Why,-1
2959,2004,60,Sean Paul featuring Sasha,I'm Still in Love with You,-1


In [281]:
uriList = []
for i in leftovers.iterrows():
    # get name of search query
    artist = i[1]['artist'].split()[0]
    song = i[1]['title']
    name = " artist:" + artist 
    result = sp.search(name,type="track", limit=50)
    
    # reset foundURI flag and search results for song
    foundURI = False
    for i in range(len(result['tracks']['items'])):
        if result['tracks']['items'][i]['name'][:6] == song[:6]:
            uri = result['tracks']['items'][i]['uri']
            foundURI = True
            break
            
    # if found append URI
    if foundURI:
        uriList.append(uri)
    else:
        uriList.append("-1")

In [286]:
uris = pd.DataFrame(uriList,columns=['uri'])
# leftovers = leftovers.drop(['uri'], axis=1)
leftovers['uri'] = uris['uri'].values
leftovers

,year,rank,artist,title,uri
23,2006,24,Lil Jon Featuring E-40 & Sean Paul Of The Youn...,Snap Yo Fingers,-1
31,2006,32,Christina Aguilera,Ain't No Other Man,spotify:track:7huo2wvrCgRucUsjdSDLQV
40,2006,41,T-Pain Featuring Mike Jones,I'm N Luv (Wit A Stripper),spotify:track:4pNlxC6xJ3PEJdf6EgS7Ke
42,2006,43,Fort Minor Featuring Holly Brook,Where'd You Go,spotify:track:7pAT4dOUzjq8Ziap5ShIqC
49,2006,50,Mariah Carey,Don't Forget About Us,spotify:track:1HkVSmzBbdZ5Bzdd00IZgh
...,...,...,...,...,...
2870,2003,71,Frankie J,Don't Wanna Try,spotify:track:1XE1UGRpvvkaFEeqe1OJpw
2884,2003,85,Young Gunz,"Can't Stop, Won't Stop",-1
2896,2003,97,Norah Jones,Don't Know Why,spotify:track:6ybViy2qrO9sIi41EgRJgx
2959,2004,60,Sean Paul featuring Sasha,I'm Still in Love with You,spotify:track:3MhdH8PxqH1FuQp3HBptUI


In [293]:
leftovers2 = leftovers[(leftovers == '-1').any(axis=1)]

C:\Users\Ambrose\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [307]:
uriList = []
for i in leftovers2.iterrows():
    # get name of search query
    artist = i[1]['artist'].split()[0]
    song = i[1]['title']
    song = " ".join(song.split())
    track = song.split()[0]
    name = "track:" + song
    result = sp.search(name,type="track", limit=50)
    
    # reset foundURI flag and search results for song
    foundURI = False
    for i in range(len(result['tracks']['items'])):
        if result['tracks']['items'][i]['name'][:6].lower() == song[:6].lower():
            uri = result['tracks']['items'][i]['uri']
            foundURI = True
            break
            
    # if found append URI
    if foundURI:
        uriList.append(uri)
    else:
        uriList.append("-1")

In [310]:
uris = pd.DataFrame(uriList,columns=['uri'])
leftovers2 = leftovers2.drop(['uri'], axis=1)
leftovers2['uri'] = uris['uri'].values
leftovers2

,year,rank,artist,title,uri
23,2006,24,Lil Jon Featuring E-40 & Sean Paul Of The Youn...,Snap Yo Fingers,spotify:track:6o3s08kk2fQI37vxGZDrJ1
51,2006,52,E-40 Featuring T-Pain & Kandi Girl,U And Dat,spotify:track:7g7M96UGiA7OR79H86gV4m
61,2006,62,Eminem Featuring Nate Dogg,Shake That,spotify:track:0fGh0trRN1xz3s41r7V5ce
99,2006,100,Teddy Geiger,For You I Will (Confidence),spotify:track:43q9QXWgnB6b4qRq60CO0N
148,2007,49,R. Kelly Or Bow Wow (Featuring T.I. & T-Pain),I'm A Flirt,-1
...,...,...,...,...,...
2768,2002,69,Angie Martinez featuring Lil' Mo and Sacario,If I Could Go!,spotify:track:056mbYcqyQo86Rfr9XfyYm
2796,2002,97,Dixie Chicks,Long Time Gone,spotify:track:0JiX8NsRdCHm8GliFuf97V
2834,2003,35,Dixie Chicks,Landslide,spotify:track:5ihS6UUlyQAfmp48eSkxuQ
2836,2003,37,Jay-Z featuring Beyoncé,'03 Bonnie & Clyde,spotify:track:3LEeO2UXO55ikhWmiXaJpK


### Why did you repeat the above code instead of creating a function?

I intended to only **have** to run that loop one time, but having over 200 stragglers creates too much work to manually grab, so I wanted to tweak the parameters for song, artist, and track each run. In order to do so I would have had to feed it a premade list with those tweaked values.

If you're using this code for your own collection purporses, if you're pulling a lot of data I highly suggest you query with a string "artist:artist name track:track name"


<hr>
Anyway, now that we got a significant amount of progress we can combine the leftover runs with the original run and then upload it to the server

We can take advantage of dropna along with concat to combine the first uri scrape with the do-over runs.

In [319]:
import numpy as np

leftovers = leftovers.replace(to_replace="-1",value=np.nan)

In [323]:
leftovers = leftovers.dropna()

In [326]:
combined = pd.concat([leftovers, leftovers2])

In [327]:
combined

,year,rank,artist,title,uri
31,2006,32,Christina Aguilera,Ain't No Other Man,spotify:track:7huo2wvrCgRucUsjdSDLQV
40,2006,41,T-Pain Featuring Mike Jones,I'm N Luv (Wit A Stripper),spotify:track:4pNlxC6xJ3PEJdf6EgS7Ke
42,2006,43,Fort Minor Featuring Holly Brook,Where'd You Go,spotify:track:7pAT4dOUzjq8Ziap5ShIqC
49,2006,50,Mariah Carey,Don't Forget About Us,spotify:track:1HkVSmzBbdZ5Bzdd00IZgh
97,2006,98,T-Pain,I'm Sprung,spotify:track:0DLOyyQvwPTSDKuhpzMMwA
...,...,...,...,...,...
2768,2002,69,Angie Martinez featuring Lil' Mo and Sacario,If I Could Go!,spotify:track:056mbYcqyQo86Rfr9XfyYm
2796,2002,97,Dixie Chicks,Long Time Gone,spotify:track:0JiX8NsRdCHm8GliFuf97V
2834,2003,35,Dixie Chicks,Landslide,spotify:track:5ihS6UUlyQAfmp48eSkxuQ
2836,2003,37,Jay-Z featuring Beyoncé,'03 Bonnie & Clyde,spotify:track:3LEeO2UXO55ikhWmiXaJpK


In [328]:
top100 = top100.replace(to_replace="-1",value=np.nan)

In [330]:
top100 = top100.dropna()

In [331]:
allCombined = pd.concat([top100, combined])

In [332]:
allCombined

,year,rank,artist,title,uri
0,2006,1,Daniel Powter,Bad Day,spotify:track:0mUyMawtxj1CJ76kn9gIZK
1,2006,2,Sean Paul,Temperature,spotify:track:0k2GOhqsrxDTAbFFSdNJjT
2,2006,3,Nelly Furtado Featuring Timbaland,Promiscuous,spotify:track:47aQT2aV12TyilaoYi1NiD
3,2006,4,James Blunt,You're Beautiful,spotify:track:7j6C8ITuzhA7GHmj2z9mSd
4,2006,5,Shakira Featuring Wyclef Jean,Hips Don't Lie,spotify:track:3ZFTkvIE7kyPt6Nu3PEa7V
...,...,...,...,...,...
2768,2002,69,Angie Martinez featuring Lil' Mo and Sacario,If I Could Go!,spotify:track:056mbYcqyQo86Rfr9XfyYm
2796,2002,97,Dixie Chicks,Long Time Gone,spotify:track:0JiX8NsRdCHm8GliFuf97V
2834,2003,35,Dixie Chicks,Landslide,spotify:track:5ihS6UUlyQAfmp48eSkxuQ
2836,2003,37,Jay-Z featuring Beyoncé,'03 Bonnie & Clyde,spotify:track:3LEeO2UXO55ikhWmiXaJpK


In [334]:
# Now we'll upload it, and find the last 43 songs by other means
# we want to keep the index becuase the values have been a bit mixed up

allCombined.to_sql('top100B', engine, if_exists='replace', index=True)

### Finding the stragglers

41/43 of the remaining songs were able to be grabbed via a playlist creation tool on soundiiz.com. This tool allowed me to feed the missing songs via CSV and their app searched for the tough to find songs. The playlist "6Bb9CAgN1zDfcDWFz87Evn" was a collection of these songs.

the last two songs "Raphael Saadiq - Ask of You" and "ESPN Presents The Jock Jam" are not on spotify

In [342]:
def show_tracks (results,uriArray):
    for i, item in enumerate(results['items']):
        track = item['track']
        uriArray.append([track['name'],track['id']])

def getTrackIDs(user, playlist_id):
    trackId= []
    playlist = sp.user_playlist(user, playlist_id)
    tracks = playlist['tracks']
    show_tracks(tracks,trackId)
    
    while tracks['next']:
        tracks = sp.next(tracks)
        show_tracks(tracks,trackId)

    return trackId

j = getTrackIDs('' , "6Bb9CAgN1zDfcDWFz87Evn")

In [350]:
df = pd.DataFrame(j, columns = ['title', 'uri']) 

In [347]:
finalReplacements = leftovers2[(leftovers2 == '-1').any(axis=1)]

C:\Users\Ambrose\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [356]:
finalReplacements.sort_values('title').head()

,year,rank,artist,title,uri
1526,1990,27,Nelson,(Can't Live Without Your) Love and Affection,-1
1660,1991,61,The KLF,3 a.m. Eternal,-1
2082,1995,83,Raphael Saadiq,Ask of You,-1
947,2015,48,Rihanna,B**** Better Have My Money,-1
1584,1990,85,D Mob featuring Cathy Dennis,C'mon and Get My Love,-1


In [355]:
df.sort_values('title').head()

,title,uri
29,(Can't Live Without Your) Love And Affection -...,3JjY2L2bqql54j1SjnjQ23
33,(I Can't Help) Falling In Love With You,7ojJ4XvqBhBcteM0zjMebT
30,3AM Eternal - Live at the S.S.L.,0ldLTYuqnHU92weLtw6W9m
26,Bitch Better Have My Money,0NTMtAO2BV4tnGvw9EgBVq
7,C'Mon And Get My Love (feat. Cathy Dennis) - 7...,2M3YldsyQfOixYWqwagtse


## Adding them to the final table

Since the values don't line up due to slight differences in the spotify and billboard names (i.e. B**** vs. Bitch), I think exporting as a csv and filling in the final values semi-manually rather than programatically is the best option.

For a larger amount of data (100+ rows) it would absolutely be worth the time setting up a function that checks titles that match (or almost match) and pair them together, but for 40 rows advanced-copy and pastes get the job done. 

<hr>

Now that URI collection is over, we can grab all the features of the 2998 top 100 songs

In [53]:
from sqlalchemy import create_engine
from pandas.io import sql
import pandas as pd 

hostname=''
dbname=''
uname=''
pwd=''

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=dbname, user=uname, pw=pwd))

#grab the table, remember we saved the one with URIs as top100B
top100 = pd.read_sql('SELECT * FROM top100B', con=engine,index_col=['index'])

In [54]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time 
import pprint as pp
import numpy as np

client_id = ''
client_secret = ''

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [55]:
# get rid of index label from sql
top100 = top100.rename_axis(None)

In [56]:
# remove two songs missing uris
top100 = top100[top100["uri"].str.contains("-1")==False]

In [57]:
top100 = top100.sort_index()

In [58]:
top100

,year,rank,artist,title,uri
0,2006,1,Daniel Powter,Bad Day,spotify:track:0mUyMawtxj1CJ76kn9gIZK
1,2006,2,Sean Paul,Temperature,spotify:track:0k2GOhqsrxDTAbFFSdNJjT
2,2006,3,Nelly Furtado Featuring Timbaland,Promiscuous,spotify:track:47aQT2aV12TyilaoYi1NiD
3,2006,4,James Blunt,You're Beautiful,spotify:track:7j6C8ITuzhA7GHmj2z9mSd
4,2006,5,Shakira Featuring Wyclef Jean,Hips Don't Lie,spotify:track:3ZFTkvIE7kyPt6Nu3PEa7V
...,...,...,...,...,...
2995,2004,96,Nelly featuring Tim McGraw,Over and Over,spotify:track:2b8QQqf8Vwb7bfz22ja8Fk
2996,2004,97,Gretchen Wilson,Redneck Woman,spotify:track:26bL4gSULWDgdIMX0pRFrG
2997,2004,98,Eminem,Just Lose It,spotify:track:0qcjuYtMWhBjXg0Xwt5SzS
2998,2004,99,John Michael Montgomery,Letters from Home,spotify:track:15cGdiAHmezm0P04jVAjMo


In [59]:
uris = top100["uri"].tolist()

In [60]:
#for song features
def getTrackFeatures(id):
    meta = sp.track(id)
    features = sp.audio_features(id)

    #meta
    ids = meta['id']
    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    release_date = meta['album']['release_date']
    length = meta['duration_ms'] #return of this will be divided by 60000 to convert from ms to minutes
    popularity = meta['popularity']

    # features
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    time_signature = features[0]['time_signature']
    key = features[0]['key']
    valence = features[0]['valence']
    


    track = [ids, name, album, artist, release_date, length/60000, popularity, danceability, acousticness, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature,key,valence]
    return track

In [61]:
# this one will take a minute or two

tracks = []

for i in uris:
    try:
        track = getTrackFeatures(i)
        tracks.append(track)
    except:
        print("invalid")
        tracks.append(i)

### with a quick peek, it looks like the songs are still in order, so we can concat these and retain the ranks

In [62]:
for i in tracks:
    print(i[1])

Bad Day
Temperature
Promiscuous
You’re Beautiful - Acoustic Cover
Hips Don't Lie (feat. Wyclef Jean)
Unwritten
Crazy
Ridin'
SexyBack (feat. Timbaland)
Check On It (feat. Bun B & Slim Thug)
Be Without You - Kendu Mix
Grillz
Over My Head (Cable Car)
Me & U
Buttons
Run It! (feat. Juelz Santana)
So Sick
It's Goin' Down (feat. Nitti)
SOS
I Write Sins Not Tragedies
Move Along
London Bridge
Dani California
Snap Yo Fingers
Lean Wit It, Rock Wit It
What Hurts The Most
How to Save a Life
Unfaithful
Chasing Cars
Lips Of An Angel
Everytime We Touch
Ain't No Other Man
Dance, Dance
Gold Digger
Money Maker
Ms. New Booty
(When You Gonna) Give It Up to Me (feat. Keyshia Cole) - Radio Version
Photograph
Because of You
Stickwitu
I'm N Luv (Wit A Stripper) 2 (feat. Pimp C, Too $hort, MJG, Twista & Paul Wall) - Tha Remix
My Humps
Where'd You Go (feat. Holly Brook & Jonah Matranga)
Yo (Excuse Me Miss)
Walk Away
Laffy Taffy
What You Know
Dirty Little Secret
Savin' Me
Don't Forget About Us - Radio Edit
Sexy L

You're Still The One
Truly Madly Deeply
How Do I Live
Together Again
All My Life
Candle In The Wind 1997
Nice & Slow
I Don't Want to Wait
How's It Going to Be
No, No, No Pt. 2 (feat. Wyclef Jean)
My Heart Will Go On - Love Theme from "Titanic"
Gettin' Jiggy Wit It
You Make Me Wanna...
My Way
My All
The First Night
Been Around the World (feat. The Notorious B.I.G. & Mase)
Adia
Crush
Everybody (Backstreet's Back) - Radio Edit
I Don't Want to Miss a Thing - From "Armageddon" Soundtrack
Body Bumpin' (Yippie-Yi-Yo)
This Kiss
I Don't Ever Want to See You Again
Let's Ride
Sex & Candy
Show Me Love - Radio Version
A Song For Mama
What You Want (feat. Total)
Frozen
Gone Till November
My Body
Tubthumping
Deja Vu (Uptown Baby)
I Want You Back - Radio Edit
When the Lights Go Out - Radio Edit
They Don't Know
Make 'Em Say Uhh - Edit
Make It Hot (feat. Missy Elliott & Mocha)
Never Ever
I Get Lonely
Feel so Good
Say It
Kiss The Rain
Come with Me Now
Romeo & Juliet
It's All About Me
I Will Come To You
O

In [63]:
features = pd.DataFrame(tracks, columns = ['ids', 'name', 'album', 'artist',
                                           'release_date', 'length', 'popularity', 
                                           'danceability', 'acousticness', 'energy', 
                                           'instrumentalness', 'liveness', 'loudness', 
                                           'speechiness', 'tempo', 'time_signature','keys','valence']) 


In [64]:
# remove the name of the song and artist, because its kind of redundant
# ill keep ids right now, but delete it after confirming they're lined up

del features['name']
del features['artist']

In [65]:
# we have to drop the indexes before concat or they wont line up properly
top100.reset_index(drop=True, inplace=True)
features.reset_index(drop=True, inplace=True)

In [66]:
combined = pd.concat([top100, features], axis=1)

In [68]:
combined.tail()

,year,rank,artist,title,uri,ids,album,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
2993,2004,96,Nelly featuring Tim McGraw,Over and Over,spotify:track:2b8QQqf8Vwb7bfz22ja8Fk,2b8QQqf8Vwb7bfz22ja8Fk,Suit,2004-09-14,253933,61,0.644,0.0065,0.517,0.000001,0.158,-10.020,0.0803,169.799,4
2994,2004,97,Gretchen Wilson,Redneck Woman,spotify:track:26bL4gSULWDgdIMX0pRFrG,26bL4gSULWDgdIMX0pRFrG,Here For The Party,2004-05-11,221333,63,0.499,0.1300,0.825,0.000000,0.306,-5.146,0.1770,185.069,4
2995,2004,98,Eminem,Just Lose It,spotify:track:0qcjuYtMWhBjXg0Xwt5SzS,0qcjuYtMWhBjXg0Xwt5SzS,Encore (Deluxe Version),2004-11-12,248680,66,0.940,0.0581,0.633,0.000040,0.281,-3.560,0.0467,121.003,4
2996,2004,99,John Michael Montgomery,Letters from Home,spotify:track:15cGdiAHmezm0P04jVAjMo,15cGdiAHmezm0P04jVAjMo,Letters From Home,2004-04-20,267533,56,0.644,0.3670,0.626,0.000006,0.155,-7.243,0.0258,128.029,4
2997,2004,100,Kenny Chesney,I Go Back,spotify:track:57I7CUbXo4XlWuTVB8pXW6,57I7CUbXo4XlWuTVB8pXW6,When The Sun Goes Down,2004-01-22,243666,58,0.532,0.0409,0.842,0.000000,0.217,-5.754,0.0503,90.187,4


### Looks like they line up, we can now delete the ids list, and upload it. Also, not 100% necessary, but I want to organize the columns.

In [69]:
del combined['ids']

In [71]:
cols = combined.columns.tolist()
cols

['year',
 'rank',
 'artist',
 'title',
 'uri',
 'album',
 'release_date',
 'length',
 'popularity',
 'danceability',
 'acousticness',
 'energy',
 'instrumentalness',
 'liveness',
 'loudness',
 'speechiness',
 'tempo',
 'time_signature']

In [72]:
cols = ['year',
 'rank',
 'artist',
 'title',
 'album',
 'release_date',
 'length',
 'popularity',
 'danceability',
 'acousticness',
 'energy',
 'instrumentalness',
 'liveness',
 'loudness',
 'speechiness',
 'tempo',
 'time_signature',
 'keys',
 'valence',
 'uri']

In [76]:
combined = combined[cols]

In [81]:
# Now that we're completely done with collection. I am going to replace
# the original top100 list with this feature-complete list
# I can also delete the top100B list as well

combined.to_sql('top100', engine, if_exists='replace', index=True)

sql = 'DROP TABLE IF EXISTS top100B;'
engine.execute(sql)

# All done!

Also, just for fun, I decided to see if it was possible to create one big playlist out of all of these songs. To my suprise, spotify supports copying and pasting URLs and URIs to create playlists. So I organized the songs by year and rank and created a playlist. So it starts at the \#1 song of 1990's and finishes at the \#100 song of 2020. Also it's over 202 hours long... 

Here's the playlist if you're interested: https://open.spotify.com/playlist/1mIpUKR4EMC65HoAJiBUFs?si=mjrqzI0nQ2u8y2BWww0rGw